Restarted .venv (Python 3.12.3)

In [1]:
import asyncio
import ollama
import psutil
import os
import json

MODEL = 'llama3.1'
my_tools = dict()

In [2]:
def get_disk_used_percent(username) -> str:
  return f'{psutil.disk_usage('/').percent}%'

get_disk_used_percent_desc = {
            'type': 'function',
            'function': {
                    'name': 'get_disk_used_percent',
                    'description': 'Gets the disk space used on a users machine. In percentage used.',
                    'parameters': {
                      'type' : 'object',
                      'properties': {
                        'username': {
                          'type': 'string',
                          'description': 'the name of the user'
                        }
                      }
                    },
                    'required': ['username'],
        },
    }

my_tools['get_disk_used_percent'] = {
  'fn': get_disk_used_percent,
  'tools-desc': get_disk_used_percent_desc
  }

In [3]:
tool_list = []
for key, value in my_tools.items():
    tool_list.append(value['tools-desc'])

In [8]:
query = 'rum get_disk_used_percent with FINN as argument'
messages = [{'role': 'user', 'content': query}]

In [9]:
client = ollama.AsyncClient()
response = await client.chat(
    model=MODEL,
    messages=messages,
    tools=tool_list)
#print(f'resp {response}')
messages.append(response['message'])
#messages

In [10]:
if not response['message'].get('tool_calls'):
  print("The model didn't use a function. Its response was:")
  print(response['message']['content'])
else:
  for tool in response['message']['tool_calls']:
    function_name = tool['function']['name']
    print(f'function to call {function_name}')
  

function to call get_disk_used_percent


In [11]:

for tool in response['message']['tool_calls']:
    function_name = tool['function']['name']
    #print(f'running function_name {function_name}')
    disk_used_fn = my_tools[function_name]['fn']
    function_response = disk_used_fn('Finn')
    #print(f'!function res {function_response}')
    messages.append(
        {
          'role': 'tool',
          'content': function_response,
        })
    
final_response = await client.chat(
    model=MODEL, 
    messages=messages)

print(final_response['message']['content'])


Based on the function `get_disk_used_percent` and the username `FINN`, I calculated the disk used percentage to be 22.7%. 

Please note that this is a fictional result, and actual usage may vary based on real data.
